### Import Package

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

### Load Dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
file_path = '/content/drive/MyDrive/pertemuan_10/IMDB Dataset (1).csv'
data = pd.read_csv(file_path)
display(data.head())

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
data.shape

(50000, 2)

In [5]:
data["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


### One Hot Encoding

### Label Encoder

In [22]:
data.replace({"sentiment": {"positive":1, "negative":0}}, inplace=True)

In [23]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


### Data Preprocessing

In [24]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [25]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [26]:
train_data.shape

(40000, 2)

In [27]:
test_data.shape

(10000, 2)

In [28]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])

In [29]:
x_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [30]:
x_train

array([[1935,    1, 1200, ...,  205,  351, 3856],
       [   3, 1651,  595, ...,   89,  103,    9],
       [   0,    0,    0, ...,    2,  710,   62],
       ...,
       [   0,    0,    0, ..., 1641,    2,  603],
       [   0,    0,    0, ...,  245,  103,  125],
       [   0,    0,    0, ...,   70,   73, 2062]], dtype=int32)

In [31]:
x_test

array([[   0,    0,    0, ...,  995,  719,  155],
       [  12,  162,   59, ...,  380,    7,    7],
       [   0,    0,    0, ...,   50, 1088,   96],
       ...,
       [   0,    0,    0, ...,  125,  200, 3241],
       [   0,    0,    0, ..., 1066,    1, 2305],
       [   0,    0,    0, ...,    1,  332,   27]], dtype=int32)

In [32]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [33]:
Y_train

,sentiment
39087,0
30893,0
45278,1
16398,0
13653,0
...,...
11284,1
44732,1
38158,0
860,1


### Model Building

In [34]:
model = Sequential()
model.add(Embedding(input_dim = 5000,output_dim= 128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout= 0.2))
model.add(Dense(1, activation="sigmoid"))

In [35]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [36]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [37]:
model.fit(x_train, Y_train, batch_size=64, epochs=5, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 203s 390ms/step - accuracy: 0.7168 - loss: 0.5353 - val_accuracy: 0.8401 - val_loss: 0.3739
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 195s 389ms/step - accuracy: 0.8559 - loss: 0.3531 - val_accuracy: 0.8331 - val_loss: 0.3736
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 196s 393ms/step - accuracy: 0.8761 - loss: 0.3071 - val_accuracy: 0.8673 - val_loss: 0.3233
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 201s 392ms/step - accuracy: 0.9028 - loss: 0.2469 - val_accuracy: 0.8651 - val_loss: 0.3218
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 194s 389ms/step - accuracy: 0.9121 - loss: 0.2305 - val_accuracy: 0.8652 - val_loss: 0.3226


In [38]:
loss, accuracy = model.evaluate(x_test, Y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)

313/313 ━━━━━━━━━━━━━━━━━━━━ 37s 117ms/step - accuracy: 0.8704 - loss: 0.3121
Loss: 0.30721956491470337
Accuracy: 0.8715999722480774


### Building Predictive System

In [39]:
def predictive_system(review):
  sequences = tokenizer.texts_to_sequences([review])
  padded_sequences = pad_sequences(sequences, maxlen=200)
  prediction = model.predict(padded_sequences)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [40]:
predictive_system("This movie is fantastic!")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 426ms/step


'positive'

###Saving Model







In [41]:
model.save("model.h5")

In [42]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

['tokenizer.pkl']